# Introduction

In [ ]:
%pip install ipynao==0.2

In [ ]:
import ipynao
from ipywidgets import Output

import asyncio
%gui asyncio

In [ ]:
w = ipynao.NaoRobotWidget()
w

### Connect to Robot

In [ ]:
w.connect("192.168.1.115", port="80")

### Call services

In [ ]:
w

In [ ]:
tts = w.service("ALTextToSpeech")

In [ ]:
tts.say("how is it going?")

In [ ]:
eyes = w.service("ALLeds")

In [ ]:
eyes.rasta(3)

### Retrieve data from the robot
Prefix method names with `async_` to be able to wait for returns

In [ ]:
w

In [ ]:
motion = w.service("ALMotion")

In [ ]:
robot_summary = asyncio.ensure_future(motion.async_getSummary())

In [ ]:
print(robot_summary.result())

In [ ]:
motion.wakeUp()

In [ ]:
motion.rest()

In [ ]:
w

In [ ]:
posture = w.service("ALRobotPosture")

In [ ]:
posture.goToPosture("StandInit", 0.5)

In [ ]:
motion.rest()

All events occur at the same time

In [ ]:
eyes.fade("AllLedsBlue", 1.0, 2)
eyes.fade("AllLedsRed", 1.0, 2)
eyes.fade("AllLedsGreen", 0, 2)

### Camera images

In [ ]:
w

In [ ]:
vid = w.service("ALVideoDevice")

In [ ]:
resolution = 0
color_space = 11
fps = 20
sub_ID = "jupy"

cam_sub = asyncio.ensure_future(vid.async_subscribeCamera(sub_ID, 0, resolution, color_space, fps))

In [ ]:
sub_name = cam_sub.result()
print(sub_name)

In [ ]:
img_future = asyncio.ensure_future(vid.async_getImageRemote(sub_name))

In [ ]:
img = img_future.result()

In [ ]:
%pip install Pillow

In [ ]:
from PIL import Image, ImageOps

wid, hei = img[:2]
shape = img[:3]
image_data = img[6]
pil_img = Image.frombytes('RGB',(wid, hei), bytes(image_data))
pil_img = ImageOps.mirror(pil_img)

In [ ]:
# Display the image
pil_img

### Awaiting tasks

In [ ]:
w

In [ ]:
animate = w.service("ALAnimationPlayer")

In [ ]:
async def little_dance():
    await motion.async_wakeUp()
    await animate.async_run("animations/Stand/Gestures/Hey_2")
    await animate.async_run("animations/Stand/Gestures/Enthusiastic_5")
    await animate.async_run("animations/Stand/Gestures/No_9")
    await motion.async_moveTo(0, 0.2, 0)
    await animate.async_run("animations/Stand/Gestures/Explain_9")
    await animate.async_run("animations/Stand/Gestures/You_4")
    await animate.async_run("animations/Stand/Gestures/Explain_11")
    await motion.async_moveTo(0, -0.2, 0)
    await animate.async_run("animations/Stand/Gestures/YouKnowWhat_6")
    await animate.async_run("animations/Stand/Gestures/No_8")
    await posture.async_goToPosture("LyingBack", 1.0)
    await posture.async_goToPosture("Stand", 1.0)
    await animate.async_run("animations/Stand/Gestures/BowShort_1")
    await motion.async_rest()

In [ ]:
asyncio.ensure_future(little_dance())